In [1]:
#import
import math
from scipy.integrate import solve_ivp
import numpy as np

from collections import deque
import json

#overall arguments
#for simulating by the scipy ODE solver
maxT = 0.1 #max time for evolution inside a rectangle
rtol=1e-7
M=2
#solve_ivp(fun, t_span, y0, method, t_eval, dense_output, events, vectorized, args, **options)
#method in ['RK23', 'RK45', 'Radau', 'BDF', 'LSODA']
Method='RK23' #method argument is not necessary
t_eval_points_count = 10 #400

In [2]:
#INPUT Van der Pol Oscillator
#input to classic QDA

#ODE for 2 variables
def vanderpol_f(t, y):
    x0,x1 = y
    return np.array([ x1, 
                      2 * ( 1 - x0 * x0 ) * x1 - x0 ])

def vanderpol_f_auton(y):
    return vanderpol_f(0,y)

#tresholds
x0min = -8
x0max = 8
x1min = -8
x1max = 8

pieces_count = 100

stepx0 = ( x0max - x0min ) / pieces_count
stepx1 = ( x1max - x1min ) / pieces_count

tresholdsX0 = np.arange ( x0min, x0max+(0.3*stepx0), stepx0 )
tresholdsX1 = np.arange ( x1min, x1max+(0.3*stepx1), stepx1 )

tresholds = [ tresholdsX0, tresholdsX1 ]

In [ ]:
#INPUT Brusselator
#input to classic QDA

a=1.0
b=1.7
#a=1.0
#b=3.0

#ODE for 2 variables
def brusselator_f(t, y):
    x0,x1 = y
    return np.array([ a + x0 * x0 * x1 - b * x0 - x0, 
                      b * x0 - x0 * x0 * x1 ])

def brusselator_f_auton(y):
    return brusselator_f(0,y)

#tresholds
x0min = 0.0
x0max = 6.0
x1min = 0.0
x1max = 6.0

pieces_count = 100

stepx0 = ( x0max - x0min ) / pieces_count
stepx1 = ( x1max - x1min ) / pieces_count

tresholdsX0 = np.arange ( x0min, x0max+(0.3*stepx0), stepx0 )
tresholdsX1 = np.arange ( x1min, x1max+(0.3*stepx1), stepx1 )

tresholds = [ tresholdsX0, tresholdsX1 ]

In [46]:
#INPUT G1/S switch
#input to classic QDA

#ODE for 2 variables
def g1s_f(t, y):
    x0,x1 = y
    return np.array([ ( x1/(0.5+x1) )*( 0.5/(0.5+x0) ) -0.011*x0, 
                      0.05 + 1.6 *( (0.0016+x1*x1)/(16.0+x1*x1) )*( 5.0/(5.0+x0) ) - 0.1*x1 ])

def g1s_f_auton(y):
    return g1s_f(0,y)

#tresholds
x0min = 0.0
x0max = 10.0
x1min = 0.0
x1max = 10.0

pieces_count = 100

stepx0 = ( x0max - x0min ) / pieces_count
stepx1 = ( x1max - x1min ) / pieces_count

tresholdsX0 = np.arange ( x0min, x0max+(0.3*stepx0), stepx0 )
tresholdsX1 = np.arange ( x1min, x1max+(0.3*stepx1), stepx1 )

tresholds = [ tresholdsX0, tresholdsX1 ]

In [42]:
#zkouska input
def g1s_f(t, y):
    x0,x1 = y
    return np.array([ ( x1/(0.5+x1) )*( 0.5/(0.5+x0) ) -0.011*x0, 
                      0.05 + 1.6 *( (0.0016+x1*x1)/(16.0+x1*x1) )*( 5.0/(5.0+x0) ) - 0.1*x1 ])

def g1s_f_auton(y):
    return g1s_f(0,y)

#tresholds
x0min = 0.0
x0max = 10.0
x1min = 0.0
x1max = 10.0

pieces_count = 3

stepx0 = ( x0max - x0min ) / pieces_count
stepx1 = ( x1max - x1min ) / pieces_count

tresholdsX0 = np.arange ( x0min, x0max+(0.3*stepx0), stepx0 )
tresholdsX1 = np.arange ( x1min, x1max+(0.3*stepx1), stepx1 )

tresholds = [ tresholdsX0, tresholdsX1 ]
print(tresholds)

[array([ 0.        ,  3.33333333,  6.66666667, 10.        ]), array([ 0.        ,  3.33333333,  6.66666667, 10.        ])]


In [3]:
#approximation by multi-affine using the derivative at vertices
#supposing a rectangle exists this finds it
def find_rectangle( x ):
    r = [0,0]
    for i in [0,1]:
        for j in range(0, pieces_count):
            if (tresholds[i][j] < x[i]):
                r[i] = j        
    return r

def approx_func_in_r_auton( x, r, func_auton ):
    a = tresholds[0][ r[0] ]
    b = tresholds[0][ r[0]+1 ]
    c = tresholds[1][ r[1] ]
    d = tresholds[1][ r[1]+1 ]
    
    v00 = [ a, c ]
    v01 = [ a, d ]
    v10 = [ b, c ]
    v11 = [ b, d ]
    
    f00 = func_auton( v00 )
    f01 = func_auton( v01 )
    f10 = func_auton( v10 )
    f11 = func_auton( v11 )
    
    coef0 = ( x[0] - v00[0] ) / ( v10[0] - v00[0] )
    coef1 = ( x[1] - v00[1] ) / ( v01[1] - v00[1] )
    
    f0c = coef0*f00 + (1-coef0)*f10
    f1c = coef0*f01 + (1-coef0)*f11
    fx = coef1*f0c + (1-coef1)*f1c
    
    return fx

def approx_func_anywhere_auton( x, func_auton ):
    r = find_rectangle( x )
    return approx_func_in_r_auton( x, r, func_auton )

def approx_func_anywhere( x, t, func_auton ):
    return approx_func_anywhere_auton( x, func_auton )

In [ ]:
#print(find_rectangle([1.0,5.0])) #01
#print( approx_func_in_r_auton( [1.0,5.0], [0,1], g1s_f_auton )) #[0.32646768 0.20964423]
#print(g1s_f_auton([1.0,5.0])) #[0.2920303  0.36306016]
#print( approx_func_anywhere_auton( [1.0,5.0], g1s_f_auton ) )#[0.32646768 0.20964423]

In [7]:
#is the rectangle inside our system of tresholds?
def exists_rectangle( r ):
    exists = True
    for i in [0,1]:
        if r[i] < 0 :
            exists = False
        elif r[i] > (pieces_count-1): #cannot start on the last treshold or higher
            exists = False
    return exists

def is_point_inside_rectangle( x, r ):
    inside = True
    for i in [0,1]:
        minim = (tresholds[i])[ r[i] ]
        maxim = (tresholds[i])[ r[i]+1 ]
        if x[i] < minim: inside = False
        if x[i] > maxim: inside = False
    return inside

#outside rectangle, below lower facet
def is_point_below_facet( x, r, direction ):
    minim_dir = (tresholds[ direction ])[ r[ direction ] ]
    return ( x[ direction ] < minim_dir)

#outside rectangle, above upper facet
def is_point_above_facet( x, r, direction ):
    maxim_dir = (tresholds[ direction ])[ r[ direction ]+1 ]
    return ( x[ direction ] > maxim_dir )

#sharp <0 with outside normal vector to facet
def RA_inside_condition( x, direction, orientation, derivative_func ):
    i_der_at_x = (derivative_func( x ))[ direction ]
    #print("deriv at x="+str(x)+" is="+str(i_der_at_x))
    if orientation == 0 : # |-> |
        #print("RA condition="+str( i_der_at_x > 0 ))
        return ( i_der_at_x > 0 )
    else: # |<-|
        #print("RA condition="+str( i_der_at_x < 0 ))
        return ( i_der_at_x < 0 )
    
#sharp >0 with outside normal vector to facet
def RA_outside_condition( x, direction, orientation, derivative_func ):
    i_der_at_x = (derivative_func( x ))[ direction ]
    if orientation == 0 : # <-| |
        return ( i_der_at_x < 0 )
    else: # | |->
        return ( i_der_at_x > 0 )

def which_facets_outside( x, r ):
    result = [ [], [] ]
    for fdir in [0,1]:
        if is_point_below_facet( x, r, fdir ):
            result[ fdir ] = [0]
        if is_point_above_facet( x, r, fdir ):
            result[ fdir ] = [1]
    return result

def exit_point_from_segment( xin, xout, r, outside_facets ):
    new_outside_facets = outside_facets
    xexit_result = xout #default value
    for i in [0,1]:
        if len( outside_facets[i] ) > 0: #i.e. len( outside_facets[i] )==1
            #find exit point on hyperplane
            xexit=[0.0,0.0] #default
            xexit[ i ] = (tresholds[i])[ r[i] + outside_facets[i][0]] #constant on facet
            coef = ( xexit[ i ] - xin[ i ] )/( xout[ i ] - xin[ i ] ) 
            xexit[ 1 - i ] = ( xout[ 1-i ] - xin[ 1-i ] ) * coef + xin[ 1-i ]
            #if in the facet (in r), let outside facets be otherwise delete this facet
            if is_point_inside_rectangle( xexit, r ):
                xexit_result = xexit
            else:
                new_outside_facets[i] = []
    return [ xexit_result, new_outside_facets ] #the exit point inside r

def real_vertices( r, fdir, fori ):
    #real coordinates of (2) vertices of a (1dim) facet
    v1 = [0,0]
    v2 = [0,0]
    v1[ fdir ] = (tresholds[ fdir ])[ r[ fdir ] + fori ]
    v2[ fdir ] = (tresholds[ fdir ])[ r[ fdir ] + fori ]
    v1[ 1-fdir ] = (tresholds[ 1-fdir ])[ r[ 1-fdir ] ]
    v2[ 1-fdir ] = (tresholds[ 1-fdir ])[ r[ 1-fdir ] + 1 ]
    return [v1,v2]

In [8]:
def approx_tiles( x, k, r, fdir ): #x is a 0dim real number
    fmin = (tresholds[ 1-fdir ])[ r[ 1-fdir ] ]
    fmax = (tresholds[ 1-fdir ])[ r[ 1-fdir ] + 1 ]
    xnorm = (x-fmin)/(fmax-fmin)
    tilemin = math.floor( k*xnorm )
    tilemax = math.ceil( k*xnorm )
    tilemin = max( tilemin, 0)
    tilemax = min( tilemax, k)
    return [ tilemin, tilemax ]

def nonempty_intersection( tiles1, tiles2 ):
    return ( max(tiles1[0],tiles2[0]) <=  min(tiles1[1],tiles2[1]) )
    
def get_intersection( tiles1, tiles2 ):
    if nonempty_intersection( tiles1, tiles2 ):
        return [ max( tiles1[0], tiles2[0] ), min( tiles1[1], tiles2[1] ) ]

def unite_tiles( tiles1, tiles2 ): #overapproximate the union by one tileset
    return [ min( tiles1[0], tiles2[0] ), max( tiles1[1], tiles2[1]) ]

def unite_exactly_tiles( tiles1, tiles2 ): #list of tilesets
    if nonempty_intersection( tiles1, tiles2 ):
        return [ unite_tiles(tiles1,tiles2) ]
    else:
        return [ tiles1, tiles2 ]
    
def is_subset_of( subs, supers ):
    if len(subs) == 0:
        return True
    else:
        return (subs[0] >= supers[0]) and (subs[1] <= supers[1])
    

def non_null_tiles( tiles_dict, k ):
    new_dict = tiles_dict
    for key in tiles_dict:
        tiles = tiles_dict[ key ]
        if len( tiles ) == 2 :
            if( tiles[0] == tiles[1] ):
                if( tiles[0] == 0 ):
                    new_dict[ key ] = [ 0, 1 ]
                elif( tiles[0] == k ):
                    new_dict[ key ] = [ k - 1, k ]
                else:
                    q = tiles[0]
                    new_dict[ key ] = [ q - 1, q + 1 ]
    return new_dict

def RA_approved_exit( exit_sets_dict, r, k, ODEfunc_auton ):
    new_dict = exit_sets_dict
    for key in exit_sets_dict:
        tiles = exit_sets_dict[ key ]
        if( len(tiles) == 2 ): #check nonempty entry sets
            if key[0] == '0': fdir = 0
            else: fdir = 1
            if key[1] == '0': fori = 0 
            else: fori = 1
            v1,v2 = real_vertices( r, fdir, fori )
            if not RA_outside_condition( v1, fdir, fori, ODEfunc_auton ) and not RA_outside_condition( v2, fdir, fori, ODEfunc_auton ):
                new_dict[ key ] = []
    return new_dict

In [9]:
#get successors routine
#input =  k ... argument for entry set approximation by (a rectangle of) tiles
#         r - rectangle ... list of int coordinates of starting tresholds (<pieces_count), 
#         e - entry set ... [entry var, entry facet], 
#                           dim-1 coordinates of tiles rectangle boundary
#         m - number of simulations per tile
#output = list of successors with entry sets (and labels)

#forward sims only for now

def get_successors( k, r, e_facet, e, ODEfunc, ODEfunc_auton ):
    e_dir,e_or = e_facet
    successors_dict = { "00":0, "01":0, "10":0, "11":0, "inside":0 }
    exit_sets_dict = { "00":[], "01":[], "10":[], "11":[], "inside":[] }
    points_to_simulate = []
        
    if len(e) == 2:
        #coordinates that are changing on the facet
        rmin_facet = (tresholds[ 1-e_dir ])[ r[1-e_dir] ]
        rmax_facet = (tresholds[ 1-e_dir ])[ r[1-e_dir] + 1 ]
        Kstep = (rmax_facet - rmin_facet)/(K*1.0) #float
        #the entry set changing coords between
        emin = rmin_facet + e[0]*Kstep
        emax = rmin_facet + e[1]*Kstep
    
        Mstep = Kstep / (M*1.0) #float
        tiles = np.arange( emin, emax, Kstep )
        #the coordinate that is constant on the facet
        e_const = (tresholds[ e_dir ])[ r[e_dir] + e_or ]
    
        #for all entry tiles simulate and count
        #ti is a beginning of tile
        for ti in tiles:
            for i in range( 0, M ):
                if e_dir == 0 :
                    x00 = e_const 
                    x01 = ti + i*Mstep
                else:
                    x00 = ti + i*Mstep
                    x01 = e_const
                points_to_simulate += [ [x00,x01] ]
    else:
        
        rmin0 = (tresholds[ 0 ])[ r[0] ]
        rmax0 = (tresholds[ 0 ])[ r[0] + 1 ]
        rmin1 = (tresholds[ 1 ])[ r[1] ]
        rmax1 = (tresholds[ 1 ])[ r[1] + 1 ]
        Kstep0 = (rmax0 - rmin0)/(K*1.0) #float
        Kstep1 = (rmax1 - rmin1)/(K*1.0) #float
            
        Mstep0 = Kstep0 / (M*1.0) #float
        Mstep1 = Kstep1 / (M*1.0) #float
        tiles0 = np.arange( rmin0, rmax0, Kstep0 )
        tiles1 = np.arange( rmin1, rmax1, Kstep1 )
    
        #for all entry tiles simulate and count
        #ti is a beginning of tile
        for ti0 in tiles0:
            for ti1 in tiles1:
                for i in range( 0, math.ceil( math.sqrt(M) ) ):
                    for l in range( 0, math.ceil( math.sqrt(M) ) ):
                        x00 = ti0 + i*Mstep0
                        x01 = ti1 + l*Mstep1
                        points_to_simulate += [ [x00,x01] ]
    
    RA_unsat_count = 0
    RA_sat_count = 0
    for x0 in points_to_simulate:
            x00 = x0[0]
            x01 = x0[1]
            #check the RA condition at x00,x01
            if RA_inside_condition( [x00,x01], e_dir, e_or, ODEfunc_auton ):
                RA_sat_count += 1
                sol = solve_ivp( vanderpol_f, [0.0,maxT], [ x00, x01 ], Method, 
                                 t_eval=np.linspace( 0, maxT, t_eval_points_count ),
                                 rtol=rtol )
                #print(sol)
                #where does this trajectory go?
                j=1
                x = [ sol.y[0][j], sol.y[1][j] ]
                #print("x0="+str([x00,x01]))
                #print("x1="+str(x))
                #print("x2="+str( [ sol.y[0][2], sol.y[1][2] ] ))
                
                while (j < (t_eval_points_count-1)) and is_point_inside_rectangle( x, r ):
                    j += 1
                    x = [ sol.y[0][j], sol.y[1][j] ]
                    #print("j="+str(j))
                if is_point_inside_rectangle( x, r ):
                    successors_dict["inside"] = result_dict["inside"] + 1
                else:
                    #where was the exit/entry point?
                    outside_facets = which_facets_outside( x, r )
                    xin = [ sol.y[0][j-1], sol.y[1][j-1] ] #last inside
                    #compute the exit point and real
                    xexit,outside_facets = exit_point_from_segment( xin, x, r, outside_facets )
                    for fdir in [0,1]:
                        if 0 in outside_facets[ fdir ]:
                            successors_dict[ str(fdir)+"0" ] += 1
                            #update exit sets:
                            newtiles = approx_tiles( xexit[1-fdir], k, r, fdir )
                            if exit_sets_dict[ str(fdir)+"0" ] == []:
                                exit_sets_dict[ str(fdir)+"0" ] = newtiles
                            else:
                                exit_sets_dict[ str(fdir)+"0" ] = unite_tiles( newtiles, exit_sets_dict[ str(fdir)+"0" ] )
                        if 1 in outside_facets[ fdir ]:
                            successors_dict[ str(fdir)+"1" ] += 1
                            newtiles = approx_tiles( xexit[1-fdir], k, r, fdir )
                            if exit_sets_dict[ str(fdir)+"1" ] == []:
                                exit_sets_dict[ str(fdir)+"1" ] = newtiles
                            else:
                                exit_sets_dict[ str(fdir)+"1" ] = unite_tiles( newtiles, exit_sets_dict[ str(fdir)+"1" ] )
            else:
                RA_unsat_count += 1

    #print( RA_sat_count )
    #print( RA_unsat_count )
    #print( successors_dict )
    #print( exit_sets_dict )
    exit_sets_dict = non_null_tiles( exit_sets_dict, k )
    #print( exit_sets_dict )
    exit_sets_dict = RA_approved_exit( exit_sets_dict, r, k, ODEfunc_auton )
    #print( exit_sets_dict )
    #result = [ successors_dict, exit_sets_dict ]
    result = [ exit_sets_dict, exitsets_dict_into_states_list( r, exit_sets_dict ) ]#successors existing rectangles
    return result

def key_into_facet( key ):
    facet = [0,0]
    if( key[0] == '0' ):
        facet[0] = 0
    else:
        facet[0]= 1
    if( key[1] == '0' ):
        facet[1] = 0
    else:
        facet[1] = 1
    return facet

def hash_f( facet ):
    return str(facet[0])+str(facet[1])

def hash_r( r ):
    hashr = str(r[0])+"|"+str(r[1])
    return hashr

def r_hash( h ):
    r = [0,0] #default value
    hparts = h.split('|')
    r[0] = int( hparts[0] )
    r[1] = int( hparts[1] )
    return r

def key_into_facet_in_successor( key ):
    facet = [0,0]
    if( key[0] == '0' ):
        facet[0] = 0
    else:
        facet[0]= 1
    if( key[1] == '0' ):
        facet[1] = 1 #the opposite than in the predecessor
    else:
        facet[1] = 0 #ditto
    return facet

def key_into_rectangle( key, r ): #key=facet of r, into successor rectangle of r
    rectangle = []
    rectangle.append(r[0])
    rectangle.append(r[1])
    facet = key_into_facet( key ) #dir,ori
    if( facet[1] == '0' ):
        rectangle[ facet[0] ] = rectangle[ facet[0] ] - 1
    else:
        rectangle[ facet[0] ] = rectangle[ facet[0] ] + 1
    return rectangle

def exitsets_dict_into_states_list( r, exit_sets_dict ):
    result = []
    for key in exit_sets_dict:
        if len(exit_sets_dict[ key ]) > 0:
            facet = key_into_facet_in_successor( key )
            rectangle = key_into_rectangle( key, r )#compute the right neighbouring rectangle
            if( exists_rectangle( rectangle )):
                result.append( [ rectangle, facet, exit_sets_dict[key] ] )
    return result

In [10]:
#use the get successors routine
K=5
succs_dictionary = get_successors( K, [60,50], [0,0], [], vanderpol_f, vanderpol_f_auton )
print(succs_dictionary)

[{'00': [], '01': [], '10': [0, 5], '11': [], 'inside': []}, [[[60, 51], [1, 1], [0, 5]]]]


In [17]:
def update_explored_for_state( explored, s, facet, entry ): #i.e. add entry
    if hash_r(s) in explored:
        new_explored_s = explored[ hash_r(s) ]
        if hash_f(facet) in explored[ hash_r(s) ]:
            #update list of explored entry sets to s through facet
            #by union with entry
            #version 0.1 = just overapproximate the union by one tileset
            #result will be list with one element (future possibly a list of more disjunct elements)
            new_explored_s[ hash_f(facet) ] = [ unite_tiles( explored[ hash_r(s) ][ hash_f(facet) ][0], entry ) ]
        else:
            #create new list and list the explored entry set
            new_explored_s[ hash_f(facet) ] = [ entry ]
    else:
        new_explored_s = { hash_f(facet) : [ entry ] }
    return new_explored_s
        

def is_already_explored( explored, s, facet, entry ):
    if hash_r(s) in explored:
        if hash_f(facet) in explored[ hash_r(s) ]:
            return is_subset_of( entry, explored[ hash_r(s) ][ hash_f(facet) ][0] )
        else:
            return False
    else:
        return False
    
#visited reflects the entry-exit relations
#version 0.1= facet to facet
#future - more nuanced entry and exit sets (i.e. more possibilities)
def update_visited_for_state( visited, s, facet, entry, exit_sets ):
    if hash_r(s) in visited:
        new_visited_s = visited[ hash_r(s) ] #dictionary to update
        if hash_f(facet) in new_visited_s: #some exit sets for this entry facet listed
            for facet_key in exit_sets:
                if len(exit_sets[ facet_key ]) > 0: #exit set != []
                    if facet_key not in new_visited_s[ hash_f(facet) ]:
                        new_visited_s[ hash_f(facet) ].append( facet_key )
        else: #exit sets for this entry facet not listed yet
            new_visited_s[ hash_f(facet) ] = []
            for facet_key in exit_sets:
                if len(exit_sets[ facet_key ]) > 0: #exit set != []
                    new_visited_s[ hash_f(facet) ].append( facet_key )
    else:#rectangle not visited yet, make new dictionary in visited
        exit_sets_list = []
        for facet_key in exit_sets:
            if len(exit_sets[ facet_key ]) > 0: #exit set != []
                exit_sets_list.append( facet_key )
        new_visited_s = { hash_f(facet) : exit_sets_list }
    return new_visited_s


#find all reachable states from one state
#e_facet: [dir,or]
#e: [0,K] or [i,j] or []=whole rectangle
def find_all_reachable_from( k, r, e_facet, e, ODEfunc, ODEfunc_auton ):
    visited = {} #2d dict of rectangle : { facet : [ list of exit facets ] }
    explored = {} #rectangle:[entry,exit]
    queue = deque([])
    queue.append( [r, e_facet, e] )
    state_counter = 0
    while len( queue )>0 :
        #print( "len Q="+str(len(queue)) )
        state_counter += 1
        if (state_counter % 1000) == 0:
            write_out( visited, explored, state_counter )
        s,facet,entry = queue.pop()
        #print( s )
        #print( explored )
        #print( visited )
        #find successors where rectangles exist
        exit_sets,succs_list = get_successors( k, s, facet, entry, ODEfunc, ODEfunc_auton )
        #print( succs_list )
        explored[ hash_r(s) ] = update_explored_for_state( explored, s, facet, entry )
        #mark successors as visited
        visited[ hash_r(s) ] = update_visited_for_state( visited, s, facet, entry, exit_sets )
        #enqueu successors
        for succ in succs_list:
            s_r,s_f,s_e = succ
            #if hash_r(s) in visited:
                #print( "visited rectangle "+str(s) )
            if not is_already_explored( explored,s_r,s_f,s_r ):
                queue.append( succ )
                #queue.append( succ ) #will be explored, i.e. its successors computed
            #else:
                #print("already explored")
    return visited

def write_out( dictionar, dictionar2, i ):
    #write content to the output file
    fo = open("./out/output"+str(i)+".txt", "w")
    fo.write("Visited:\n")
    fo.write( str(dictionar) )
    fo.write( "\n\nExplored:\n" )
    fo.write( str(dictionar2) )
    fo.close()

In [21]:
K=3
visited_vanderpol = find_all_reachable_from(  K, [60,50], [0,0], [0,K], vanderpol_f, vanderpol_f_auton )

In [23]:
print(visited_vanderpol)

write_out( visited_vanderpol, {}, "Konec" )


{'60|50': {'00': ['10']}, '60|51': {'11': ['10']}, '60|52': {'11': ['10']}, '60|53': {'11': ['10']}, '60|54': {'11': ['10']}, '60|55': {'11': ['10']}, '60|56': {'11': ['01', '10']}, '60|57': {'11': ['01', '10']}, '60|58': {'11': ['01', '10']}, '60|59': {'11': ['01', '10']}, '60|60': {'11': ['01', '10']}, '60|61': {'11': ['01', '10']}, '60|62': {'11': ['01', '10']}, '60|63': {'11': ['01', '10']}, '60|64': {'11': ['01', '10']}, '60|65': {'11': ['01', '10']}, '60|66': {'11': ['01', '10']}, '60|67': {'11': ['01', '10']}, '60|68': {'11': ['01', '10']}, '60|69': {'11': ['01', '10']}, '60|70': {'11': ['01', '10']}, '60|71': {'11': ['01', '10']}, '60|72': {'11': ['01', '10']}, '60|73': {'11': ['01', '10']}, '60|74': {'11': ['01', '10']}, '60|75': {'11': ['01', '10']}, '60|76': {'11': ['01', '10']}, '60|77': {'11': ['01', '10']}, '60|78': {'11': ['01', '10']}, '60|79': {'11': ['01', '10']}, '60|80': {'11': ['01', '10']}, '60|81': {'11': ['01', '10']}, '60|82': {'11': ['01', '10']}, '60|83': {'1

In [20]:
def minus_func( func ):
    return lambda x, t : -func(x,t)

def minus_func_auton( func ):
    return lambda x : -func(x)

#find all backwards reachable states from one state
K=2
visited_vanderpol_backwards = find_all_reachable_from(  K, [60,50], [1,0], [0,K], minus_func(vanderpol_f), minus_func_auton(vanderpol_f_auton) )

In [ ]:
https://realpython.com/how-to-implement-python-stack/
https://realpython.com/how-to-implement-python-stack/#python-stacks-and-threading
https://realpython.com/intro-to-python-threading/

In [35]:
print(visited_vanderpol_backwards)


write_out( visited_vanderpol_backwards, {}, "KonecBackwards" )

{'60|50': {'10': ['00']}, '61|50': {'01': []}}


In [ ]:
#Find the SCC around a given pivot state
#by finding an intersection of Forwards and Backwards reachability from pivot

def intersect_FB( visitedF, visitedB ):
    result = {} #will be stored as dictionary rectangle : entryF facet : list exitF facets in the forward direction
    for rkey in visitedF:
        if rkey in visitedB:
            ffacets = visitedF[ rkey ]
            bfacets = visitedB[ rkey ]
            #compare, store the "intersection"
            intersectionF = {} #in the form of entry facet F : list of exit facets F
            for fkey in ffacets:
                #if fkey somewhere in list of some bfkey
                for bfkey in bfacets:
                    if fkey in bfacets[ bfkey ]:
                        #store

In [27]:
#visualization of a set of rectangles in 2d
#import seaborn as sns
#import json
  
# reading the data from the file
#with open('./out/outputKonec.txt') as f:
#    data = f.read()
#    f.close()
#print("Data type before reconstruction : ", type(data))
# reconstructing the data as a dictionary
#js = json.loads(data)
#print("Data type after reconstruction : ", type(js))
#print(js)
#decode and visualize the keys

to_visualize_vanderpol = {'60|50': {'00': ['10']}, '60|51': {'11': ['10']}, '60|52': {'11': ['10']}, '60|53': {'11': ['10']}, '60|54': {'11': ['10']}, '60|55': {'11': ['10']}, '60|56': {'11': ['01', '10']}, '60|57': {'11': ['01', '10']}, '60|58': {'11': ['01', '10']}, '60|59': {'11': ['01', '10']}, '60|60': {'11': ['01', '10']}, '60|61': {'11': ['01', '10']}, '60|62': {'11': ['01', '10']}, '60|63': {'11': ['01', '10']}, '60|64': {'11': ['01', '10']}, '60|65': {'11': ['01', '10']}, '60|66': {'11': ['01', '10']}, '60|67': {'11': ['01', '10']}, '60|68': {'11': ['01', '10']}, '60|69': {'11': ['01', '10']}, '60|70': {'11': ['01', '10']}, '60|71': {'11': ['01', '10']}, '60|72': {'11': ['01', '10']}, '60|73': {'11': ['01', '10']}, '60|74': {'11': ['01', '10']}, '60|75': {'11': ['01', '10']}, '60|76': {'11': ['01', '10']}, '60|77': {'11': ['01', '10']}, '60|78': {'11': ['01', '10']}, '60|79': {'11': ['01', '10']}, '60|80': {'11': ['01', '10']}, '60|81': {'11': ['01', '10']}, '60|82': {'11': ['01', '10']}, '60|83': {'11': ['01', '10']}, '60|84': {'11': ['01', '10']}, '60|85': {'11': ['01', '10']}, '60|86': {'11': ['01', '10']}, '60|87': {'11': ['01', '10']}, '60|88': {'11': ['01', '10']}, '60|89': {'11': ['01', '10']}, '60|90': {'11': ['01', '10']}, '60|91': {'11': ['01', '10']}, '60|92': {'11': ['01', '10']}, '60|93': {'11': ['01', '10']}, '60|94': {'11': ['01', '10']}, '60|95': {'11': ['01', '10']}, '60|96': {'11': ['01', '10']}, '60|97': {'11': ['01', '10']}, '60|98': {'11': ['01', '10']}, '60|99': {'11': ['01', '10']}, '61|99': {'00': ['10'], '11': ['10', '01']}, '61|98': {'00': ['10'], '11': ['10', '01']}, '61|97': {'00': ['10'], '11': ['10', '01']}, '62|99': {'00': ['10'], '11': ['10']}, '61|96': {'00': ['10'], '11': ['10', '01']}, '62|98': {'00': ['10'], '11': ['10']}, '61|95': {'00': ['10'], '11': ['10', '01']}, '62|97': {'00': ['10'], '11': ['10']}, '61|94': {'00': ['10'], '11': ['10', '01']}, '62|96': {'00': ['10'], '11': ['10']}, '61|93': {'00': ['10'], '11': ['10', '01']}, '62|95': {'00': ['10'], '11': ['10']}, '61|92': {'00': ['10'], '11': ['10', '01']}, '62|94': {'00': ['10'], '11': ['10']}, '61|91': {'00': ['10'], '11': ['10', '01']}, '62|93': {'00': ['10'], '11': ['10']}, '61|90': {'00': ['10'], '11': ['10', '01']}, '62|92': {'00': ['10'], '11': ['10']}, '61|89': {'00': ['10'], '11': ['10', '01']}, '62|91': {'00': ['10'], '11': ['10']}, '61|88': {'00': ['10'], '11': ['10', '01']}, '62|90': {'00': ['10'], '11': ['10']}, '61|87': {'00': ['10'], '11': ['10', '01']}, '62|89': {'00': ['10'], '11': ['10']}, '61|86': {'00': ['10'], '11': ['10', '01']}, '62|88': {'00': ['10'], '11': ['10']}, '61|85': {'00': ['10'], '11': ['10', '01']}, '62|87': {'00': ['10'], '11': ['10']}, '61|84': {'00': ['10'], '11': ['10', '01']}, '62|86': {'00': ['10'], '11': ['10']}, '61|83': {'00': ['10'], '11': ['10', '01']}, '62|85': {'00': ['10'], '11': ['10']}, '61|82': {'00': ['10'], '11': ['10', '01']}, '62|84': {'00': ['10'], '11': ['10']}, '61|81': {'00': ['10'], '11': ['10', '01']}, '62|83': {'00': ['10'], '11': ['10']}, '61|80': {'00': ['10'], '11': ['10', '01']}, '62|82': {'00': ['10'], '11': ['10']}, '61|79': {'00': ['10'], '11': ['10', '01']}, '62|81': {'00': ['10'], '11': ['10']}, '61|78': {'00': ['10'], '11': ['10', '01']}, '62|80': {'00': ['10'], '11': ['10']}, '61|77': {'00': ['10'], '11': ['10', '01']}, '62|79': {'00': ['10'], '11': ['10']}, '61|76': {'00': ['10'], '11': ['10', '01']}, '62|78': {'00': ['10'], '11': ['10']}, '61|75': {'00': ['10'], '11': ['10', '01']}, '62|77': {'00': ['10'], '11': ['10']}, '61|74': {'00': ['10'], '11': ['10', '01']}, '62|76': {'00': ['10'], '11': ['10']}, '61|73': {'00': ['10'], '11': ['10', '01']}, '62|75': {'00': ['10'], '11': ['10']}, '61|72': {'00': ['10'], '11': ['10', '01']}, '62|74': {'00': ['10'], '11': ['10']}, '61|71': {'00': ['10'], '11': ['10', '01']}, '62|73': {'00': ['10'], '11': ['10']}, '61|70': {'00': ['10'], '11': ['10', '01']}, '62|72': {'00': ['10'], '11': ['10']}, '61|69': {'00': ['10'], '11': ['10', '01']}, '62|71': {'00': ['10'], '11': ['10']}, '61|68': {'00': ['10'], '11': ['10', '01']}, '62|70': {'00': ['10'], '11': ['10']}, '61|67': {'00': ['10'], '11': ['10', '01']}, '62|69': {'00': ['10'], '11': ['10']}, '61|66': {'00': ['10'], '11': ['10']}, '62|68': {'00': ['10'], '11': ['10']}, '61|65': {'00': ['10'], '11': ['10']}, '62|67': {'00': ['10']}, '61|64': {'00': ['10'], '11': ['10']}, '61|63': {'00': ['10'], '11': ['10']}, '61|62': {'00': ['10'], '11': ['10']}, '61|61': {'00': ['10'], '11': ['10']}, '61|60': {'00': ['10'], '11': ['10']}, '61|59': {'00': ['10'], '11': ['10']}, '61|58': {'00': ['10'], '11': ['10']}, '61|57': {'00': ['10'], '11': ['10']}, '61|56': {'00': ['10']}} 


NameError: name 'sns' is not defined